In [1]:
import json
from collections import Counter


#Validation Set
with open('val.json') as json_file:
    val = json.load(json_file)
episodes = val['episodes']
type(episodes)

list

In [2]:
question_vocab_val = val['question_vocab']
answer_vocab_val = val['answer_vocab']

In [3]:
# Check for Unknowns
qs = [item['question'] for item in episodes]
qt = [item['question_text'] for item in qs]

for q in qt:
    if '<unk>' in q:
        print(q)
        
ans = [item['answer_text'] for item in qs]
for a in ans:
    if '<unk>' in a:
        print(a)

In [4]:
#Unique Answers
unique_as = set(a for a in ans)
unique_as

{'bedroom',
 'black',
 'blue',
 'brown',
 'closet',
 'family room',
 'green',
 'grey',
 'gym',
 'hallway',
 'kitchen',
 'laundry room',
 'living room',
 'lounge',
 'off-white',
 'olive green',
 'red',
 'red brown',
 'silver',
 'slate grey',
 'spa',
 'tan',
 'white',
 'yellow'}

In [5]:
#Train Set
with open('train.json') as json_file:
    train = json.load(json_file)
train_episodes = train['episodes']
type(train_episodes)

list

In [6]:
answer_vocab_train = train['answer_vocab']
question_vocab_train = train['question_vocab']

In [7]:
# Check for Unknowns
tqs = [item['question'] for item in train_episodes]
tqt = [item['question_text'] for item in tqs]


for tq in tqt:
    if '<unk>' in tq:
        print(tq)
        
tans = [item['answer_text'] for item in tqs]
for ta in tans:
    if '<unk>' in ta:
        print(ta)

In [8]:
#Unique Answers
unique_tas = set(ta for ta in tans)
unique_tas

{'bathroom',
 'bedroom',
 'black',
 'blue',
 'brown',
 'closet',
 'dining room',
 'family room',
 'foyer',
 'green',
 'grey',
 'hallway',
 'kitchen',
 'laundry room',
 'light blue',
 'living room',
 'lounge',
 'off-white',
 'office',
 'olive green',
 'orange yellow',
 'purple',
 'purple pink',
 'red',
 'red brown',
 'silver',
 'slate grey',
 'spa',
 'tan',
 'tv room',
 'white',
 'yellow',
 'yellow green',
 'yellow pink'}

In [9]:
val_only = set(unique_as - unique_tas)
val_only

{'gym'}

In [10]:
# Get Percentages of Question Types
tqtype = [item['question_type'] for item in tqs]
q_num = len(tqtype)
c = Counter(tqtype)
type_percentages = [(i, c[i] / q_num * 100.0) for i in c]
type_percentages

[('color_room', 69.85908141962422),
 ('color', 15.91858037578288),
 ('location', 14.222338204592901)]

In [12]:
tatype = [item['question_type'] for item in qs]
q_num = len(tatype)
c = Counter(tatype)
type_percentages = [(i, c[i] / q_num * 100.0) for i in c]
type_percentages

[('color_room', 68.46153846153847),
 ('color', 17.692307692307693),
 ('location', 13.846153846153847)]

In [ ]:
#Making a mini-version of the dataset for local testing
episodes[0]

In [ ]:
# matterport scenes to use in mini-dataset
scenes = ['mp3d/1LXtFkjw3qL/1LXtFkjw3qL.glb', 'mp3d/1pXnuDYAj8r/1pXnuDYAj8r.glb', 'mp3d/2azQ1b91cZZ/2azQ1b91cZZ.glb', 'mp3d/2n8kARJN3HM/2n8kARJN3HM.glb']

In [ ]:
mini_val_episodes = list(filter(lambda d: d['scene_id'] in scenes, episodes))
len(mini_val_episodes)

In [ ]:
mini_val = {'episodes': mini_val_episodes, 'question_vocab': question_vocab_val, 'answer_vocab': answer_vocab_val}

In [ ]:
mini_train_episodes = list(filter(lambda d: d['scene_id'] in scenes, train_episodes))
mini_train = {'episodes': mini_train_episodes, 'question_vocab': question_vocab_train, 'answer_vocab': answer_vocab_train}

In [ ]:
len(mini_train_episodes)

In [ ]:
#write to json files

with open('mini_train.json', 'w') as f:
    json.dump(mini_train, f)
    
with open('mini_val.json', 'w') as f:
    json.dump(mini_val, f)